In [4]:
import numpy as np

## Задача 3
#### Даны два numpy массива:
1) массив чисел в формате double (например, np.array([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]))

2) массив порогов в формате double (например, np.array([0.1, 1.2, 1.5, 3.8, 7.7]))

#### Написать функцию, которая принимает на вход два массива (массив чисел, и массив порогов) и возвращает на выходе массив элементов, каждый из которых представляет собой сумму всех элементов исходного массива, больших или равных заданному порогу. Как бы вы решили данную задачу без использования циклов?

In [193]:
arr_numbers    = np.array([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0])
arr_thresholds = np.array([0.1, 1.2, 1.5, 3.8, 7.7])

### Версия 1
### Примечание: данный вариант неоптимален и использует циклы, его я решил привести для сравнения с другими вариантами

In [185]:
# Неоптимальный вариант с использованием циклов
# @param {numpy.ndarray} a_numbers    - массив чисел
# @param {numpy.ndarray} a_thresholds - массив порогов
#
# return {numpy.ndarray}              - массив из сумм элементов, >= заданному порогу
def sum_elems_ver1(a_numbers, a_thresholds):
    arr_sums = []
    
    for t in a_thresholds:
        curr_sum = 0
        
        for num in a_numbers:
            if num >= t:
                curr_sum += num
        
        arr_sums.append(curr_sum)
    
    return np.array(arr_sums)

In [524]:
test_ver1 = sum_elems_ver1(arr_numbers, arr_thresholds)
test_ver1

array([36., 35., 35., 30.,  8.])

### Версия 2
### Можно чуть оптимизировать этот вариант, добавив предварительную сортировку для массива чисел (в порядке убывания)

In [213]:
# Чуть более оптимальный вариант с предварительной сортировкой
# В функции создаётся отсортированная копия, но можно сортировать и исходный объект (зависит от условий задачи)
#
# @param {numpy.ndarray} a_numbers    - массив чисел
# @param {numpy.ndarray} a_thresholds - массив порогов
#
# return {numpy.ndarray}              - массив из сумм элементов, >= заданному порогу
def sum_elems_ver2(a_numbers, a_thresholds):

    arr_sums = []
    # Сортируем массив чисел в обратном порядке - создаётся его отсортированная копия
    # Можно без копирования, вызывав a_numbers[::-1].sort() - память сэкономим, но изменится исходный массив
    a_numbers_sorted = np.sort(a_numbers)[::-1]
    
    for t in a_thresholds:
        curr_sum = 0
        
        for num in a_numbers_sorted:
            if num < t:
                break
            else:
                curr_sum += num
        
        arr_sums.append(curr_sum)
    
    return np.array(arr_sums)

In [526]:
test_ver2 = sum_elems_ver2(arr_numbers, arr_thresholds)
test_ver2

array([36., 35., 35., 30.,  8.])

In [534]:
np.allclose(test_ver1, test_ver2)

True

### Версия 3
### Уберём циклы python, замедляющие работу.
### За счёт чего? - за счёт свойств матричного умножения

1. Мы можем получить матрицу-маску. Количество строк этой матрицы совпадает с количеством порогов. Строки матрицы содержат элементы со значением True для тех чисел, которые >= соответствующему порогу (в первой строке для первого порога, во второй строке - для второго порога и так далее...).
2. Получив эту матрицу (состоящую из 0 и 1), мы можем умножить её на вектор-столбец исходных чисел. Таким образом, при умножении каждой строки матрицы-маски на вектор чисел мы получаем их сумму с коэффициентами 0 или 1 (0 для чисел, которые не должны учитывться в сумме и меньше заданного порога, 1 - наоборот). И так для каждого порога.
3. На выходе получаем вектор-столбец сумм и транспонируем его, чтобы вернуть обычный массив.

In [516]:
# Вариант с использованием приёмов библиотеки numpy
#
# @param {numpy.ndarray} a_numbers    - массив чисел
# @param {numpy.ndarray} a_thresholds - массив порогов
#
# return {numpy.ndarray}              - массив из сумм элементов, >= заданному порогу
def sum_elems_ver3(a_numbers, a_thresholds):
    
    # Получаем матрицу-маску
    mask_matr = a_numbers >= a_thresholds.reshape(-1,1)
    
    # Возвразаем транспонированный вектор-столбец с суммами для соответствующих порогов
    return np.dot(mask_matr, a_numbers.transpose()).reshape(1,-1)

In [527]:
test_ver3 = sum_elems_ver3(arr_numbers, arr_thresholds)
test_ver3

array([[36., 35., 35., 30.,  8.]])

In [533]:
np.allclose(test_ver1, test_ver3)

True

### Попробуем потестировать написанные методы на случайно заполненных массивах бОльших размерностей

In [300]:
# Генерация массивов из случайных чисел
arr_numbers_2 = np.random.uniform(low=0.1, high=1000, size=(1000,))
arr_thresholds_2 = np.random.uniform(low=0.1, high=1000, size=(1000,))

In [302]:
%%timeit
# Земер времени выполнения для первой версии
sum_elems_ver1(arr_numbers_2, arr_thresholds_2)

155 ms ± 328 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [303]:
%%timeit
# Земер времени выполнения для второй версии
sum_elems_ver2(arr_numbers_2, arr_thresholds_2)

99.1 ms ± 223 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [518]:
%%timeit
# Земер времени выполнения для третьей версии
sum_elems_ver3(arr_numbers_2, arr_thresholds_2)

4.08 ms ± 121 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [536]:
# Сравним результаты 1 и 2 версий между собой
np.allclose(sum_elems_ver1(arr_numbers_2, arr_thresholds_2),sum_elems_ver2(arr_numbers_2, arr_thresholds_2))

True

In [535]:
# Сравним результаты 1 и 3 версий между собой
np.allclose(sum_elems_ver1(arr_numbers_2, arr_thresholds_2),sum_elems_ver3(arr_numbers_2, arr_thresholds_2))

True